In [10]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis
!python -m spacy download en_core_web_sm
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-06-14 07:49:33.089443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [79]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import random

In [80]:
def preprocess_text(text):
    # Tokenization
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

In [99]:
def apply_nlp_techniques(dataset):
    preprocessed_requirements = []
    requirement_types = []

    nlp = spacy.load("en_core_web_sm")  # Load the English model for spaCy

    for line in dataset:
        # Sentence Segmentation
        sentences = sent_tokenize(line)

        # Extract requirements and their types
        for sentence in sentences:
            if sentence.startswith(('F:', 'A:', 'FT:', 'L:', 'LF:', 'MN:', 'O:', 'PE:', 'PO:', 'SC:', 'SE:', 'US:')):
                requirement = sentence.split(':')[1].strip()
                requirement_type = sentence.split(':')[0].strip()
                preprocessed_requirement = preprocess_text(requirement)

                preprocessed_requirements.append(preprocessed_requirement)
                requirement_types.append(requirement_type)

                # Named Entity Recognition (NER)
                doc = nlp(sentence)
                entities = [ent.text for ent in doc.ents]

                # Part-of-Speech Tagging (POS)
                tokens = nltk.word_tokenize(sentence)
                pos_tags = nltk.pos_tag(tokens)

                # Chunking
                grammar = r"NP: {<DT>?<JJ>*<NN>}"
                cp = nltk.RegexpParser(grammar)
                tree = cp.parse(pos_tags)

                # Dependency Parsing
                dependency_tree = [(token.text, token.dep_) for token in doc]

                # Word frequency
                word_frequency = nltk.FreqDist(tokens)

    # Vectorize the preprocessed requirements
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(preprocessed_requirements)

    # Train a random forest classifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)

    # Perform cross-validation to evaluate accuracy
    cv_scores = cross_val_score(classifier, X, requirement_types, cv=5)
    mean_cv_accuracy = cv_scores.mean()
    print(f"Mean Cross-Validation Accuracy: {mean_cv_accuracy}")

    # Fit the classifier on the entire dataset
    classifier.fit(X, requirement_types)

    # Predict the requirement types
    random.shuffle(dataset)  # Shuffle the dataset randomly
    num_examples = 10  # Specify the number of random examples to check

    # Select random requirements from the dataset
    random_examples = random.sample(dataset, num_examples)
    preprocessed_random_examples = [preprocess_text(req) for req in random_examples]
    X_random = vectorizer.transform(preprocessed_random_examples)
    predicted_types = classifier.predict(X_random)

    # Print the predicted requirement types for the random examples
    for i in range(num_examples):
        print(f"Requirement: {random_examples[i]}")
        print(f"Predicted Type: {predicted_types[i]}")
        print("------------------------------")

    # Calculate and print the accuracy score
    X_all = vectorizer.transform(preprocessed_requirements)
    all_predicted_types = classifier.predict(X_all)
    accuracy = accuracy_score(requirement_types, all_predicted_types)
    print(f"Accuracy Score: {accuracy}")


In [100]:
def load_dataset(dataset_file_path):
    # Load the dataset from the file and return it
    dataset = []
    with open(dataset_file_path, 'r') as file:
        for line in file:
            dataset.append(line.strip())
    return dataset

In [103]:
def main():
    train_dataset_file_path = 'nfr.txt'
    test_dataset_file_path = 'test.txt'

    train_dataset = load_dataset(train_dataset_file_path)
    test_dataset = load_dataset(test_dataset_file_path)

    # Apply NLP techniques on the training dataset
    apply_nlp_techniques(train_dataset)

    # Apply NLP techniques on the test dataset
    apply_nlp_techniques(test_dataset)

if __name__ == '__main__':
    main()

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Mean Cross-Validation Accuracy: 0.5288449163449164
Requirement: O:The System shall not require additional third party licenses resulting in royalty fees.
Predicted Type: O
------------------------------
Requirement: LF: The website should have an African feel  but should not alienate non-Africans.  The website should use animation on pages which are describing the services  to grab the users attention and encourage them to sign up.
Predicted Type: LF
------------------------------
Requirement: F:The Disputes System shall provide view access capability for authorized users of the application.
Predicted Type: F
------------------------------
Requirement: PE:System shall let customers register on the website as a �unlimited movie subscriber� user in under 10 minutes.
Predicted Type: PE
------------------------------
Requirement: F: At the start of each turn  the product shall notify each player of his or her status.
Predicted Type: F
------------------------------
Requirement: F:System sh

# New Section